In [1]:
%autosave 0

Autosave disabled


In [7]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import StratifiedKFold

In [3]:
df = pd.read_csv('prepped_data.csv')

train_labels = pd.read_csv('../../data/prepared/train_labels.csv')
val_labels = pd.read_csv('../../data/prepared/val_labels.csv')

In [4]:
all_labels = pd.concat([train_labels, val_labels], axis=0)

In [5]:
df = df.merge(all_labels, how='left', on='customer_ID')
df.head()

,customer_ID,P_2_min,P_2_max,P_2_median,P_2_std,P_2_last,P_2_change,D_39_min,D_39_max,D_39_median,...,D_137_nulls,D_138_nulls,D_139_nulls,D_140_nulls,D_141_nulls,D_142_nulls,D_143_nulls,D_144_nulls,D_145_nulls,target
0,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,0.868580,0.960384,0.938469,0.024194,0.934745,-0.003724,0.001082,0.091505,0.002483,...,0.446319,0.446319,-0.283183,-0.219693,-0.283183,0.596403,-0.283183,-0.219331,-0.283183,0
1,0005ef1480ffe4f0e5908eae07db8e05f2e705bca6a57d...,0.068484,0.186780,0.127632,0.083648,0.068484,-0.118296,0.031711,0.914384,0.473048,...,-3.166442,-3.166442,-0.283183,-0.219693,-0.283183,-1.582977,-0.283183,-0.219331,-0.283183,1
2,0005f2ab0cc352cd64eac9e60b981ed8c3289d20fa5fce...,0.837273,0.946487,0.894418,0.030962,0.875766,-0.018652,0.004118,0.740627,0.413056,...,0.446319,0.446319,-0.283183,-0.219693,-0.283183,0.596403,-0.283183,-0.219331,-0.283183,0
3,0007ee1ac8edb6be588c6c8a158d2268e3228c49cf5277...,0.365222,0.646557,0.417593,0.081273,0.410590,-0.235967,0.000303,0.213325,0.006578,...,0.446319,0.446319,-0.283183,-0.219693,-0.283183,0.596403,-0.283183,-0.219331,-0.283183,0
4,0008ef32824d3067b4d8ab70e7c79fa04c808f4d99194f...,0.752942,0.867515,0.843999,0.030110,0.850308,0.097366,0.000355,0.009881,0.007834,...,0.446319,0.446319,1.033397,-0.219693,1.033397,0.596403,1.033397,-0.219331,1.033397,0


In [9]:
matrix = xgb.DMatrix(df.drop(columns=['customer_ID', 'target']), label=df['target'])

In [14]:
seed = 42

skf = StratifiedKFold(n_splits=5)

params = {
    'max_depth': 4,
    'learning_rate': 0.1,
    'colsample_bytree': 0.8,
    'subsample': 0.8,
    'min_child_weight': 10,
    'objective': 'binary:logistic',
    'random_state': seed
}

In [15]:
xgb.cv(params, matrix, num_boost_round=2_000, nfold=5,
       folds=skf, metrics='logloss', early_stopping_rounds=32,
       verbose_eval=True, seed=seed)

[0]	train-logloss:0.63257+0.00007	test-logloss:0.63264+0.00011
[1]	train-logloss:0.58314+0.00050	test-logloss:0.58337+0.00059


KeyboardInterrupt: 